In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# imports 
import numpy as np
import pandas as pd 
import os,random,gc
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler, MinMaxScaler,MaxAbsScaler,RobustScaler

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# variables
TRAIN_PATH = "../input/tabular-playground-series-may-2022/train.csv"
TEST_PATH = "../input/tabular-playground-series-may-2022/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-may-2022/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "id"
TARGET = "target"

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything()


In [3]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max  np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max  np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max  np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max  np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max  np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max  np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df


SyntaxError: invalid syntax (3839452742.py, line 10)

In [4]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)


In [5]:
train = reduce_memory_usage(train)
train = train.sample(10000)
gc.collect()
test = reduce_memory_usage(test)
gc.collect()


NameError: name 'reduce_memory_usage' is not defined

In [6]:
train.describe(include="O")


,f_27
count,800000
unique,670919
top,BBBBBBCJBC
freq,10


In [7]:
train["f_27"]


0         ACBADABECB
1         BBBCAAAFDE
2         BDAEAABICD
3         BAABFADDCA
4         AABFBBEMHC
             ...    
799995    ACBCBBDAAB
799996    BDBBBBDSEA
799997    BBBECADQCC
799998    AGBBAABBDE
799999    BBBBAAAGBC
Name: f_27, Length: 800000, dtype: object

In [8]:
y = train[TARGET]
X = train.drop([ID,TARGET,"f_27"],axis=1)
X_test = test.drop([ID,"f_27"],axis=1)


In [9]:
MODEL_TREE_METHOD = 'gpu_hist'

model = XGBRegressor(tree_method=MODEL_TREE_METHOD) 
model.fit(X, y)


XGBoostError: [22:12:02] /workspace/src/tree/updater_gpu_hist.cu:781: Exception in gpu_hist: [22:12:02] /workspace/src/tree/updater_gpu_hist.cu:787: Check failed: ctx_->gpu_id >= 0 (-1 vs. 0) : Must have at least one device
Stack trace:
  [bt] (0) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0xb27f2a) [0x7f88c7404f2a]
  [bt] (1) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0xb3e95a) [0x7f88c741b95a]
  [bt] (2) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0xb483cd) [0x7f88c74253cd]
  [bt] (3) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x460c79) [0x7f88c6d3dc79]
  [bt] (4) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x46176c) [0x7f88c6d3e76c]
  [bt] (5) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x4c54f7) [0x7f88c6da24f7]
  [bt] (6) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x70) [0x7f88c6a3eef0]
  [bt] (7)

In [10]:
sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
sub[TARGET] =np.round(model.predict(X_test),2)
sub.to_csv(SUBMISSION_PATH, index=False)
sub.head()


NotFittedError: need to call fit or load_model beforehand